# prepare_protein — Run a batch protein preperation in parallel

See [the tutorial](/Tutorials/batch_run-protein-prep.ipynb).

In [ ]:
import os
import asyncio
from glob import glob
from datetime import datetime
from pathlib import Path

from pdbtools import (
    pdb_fetch,
    pdb_delhetatm,
    pdb_selchain,
    pdb_rplresname,
    pdb_keepcoord,
    pdb_selresname,
)
import py3Dmol

import rush

# Set our token - ensure you have exported RUSH_TOKEN in your shell; or just
# replace the os.getenv with your token
TOKEN = os.getenv("RUSH_TOKEN")
URL = os.getenv("RUSH_URL")

# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "batch-prep"]
WORK_DIR = Path.home() / "qdx" / "rush-py-batch-prep"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

TARGET="NIX_SSH_2"

# |hide_line
if WORK_DIR.exists():
# |hide_line
    client = rush.Provider(workspace=WORK_DIR)
# |hide_line
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    access_token=TOKEN, url=URL, batch_tags=TAGS
)

# |hide_line
client = await rush.build_provider_with_functions(access_token=TOKEN, url=URL, workspace=WORK_DIR, batch_tags=TAGS, restore_by_default=True)

# Fetch datafiles
for pdb in ["1B39", "4QXI", "8FSU"]:
    complex = list(pdb_fetch.fetch_structure(pdb))
    protein = pdb_delhetatm.remove_hetatm(pdb_selchain.select_chain(complex, "A"))

    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb}_protein.pdb", "w") as f:
        for l in protein:
            f.write(str(l))

# Prepare each protein
proteins = map(lambda x: Path(x), glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")))

protein_outputs = []

for protein_path in proteins:
    print(protein_path)
    name = protein_path.stem
    (prepped_protein_qdxf, prepped_protein_pdb) = await client.prepare_protein(
        protein_path,
        tags=[name],
        target=TARGET
    )
    protein_outputs.append((name, prepped_protein_qdxf, prepped_protein_pdb))

protein_outputs

# Report progress
status = await client.status(group_by="path")
print(f"{'Module':<32} | {'Status':<32} | Count")
print("-" * 75)
for module, (status, path, count) in status.items():
    print(f"{path:<32} | {status:<32} | {count:>5}")

# Download results
await asyncio.gather(
    *[
        output[1].download(filename=f"protein_{output[0]}_prepared.qdxf.json")
        for output in protein_outputs
    ] + [
        output[2].download(filename=f"protein_{output[0]}_prepared.pdb")
        for output in protein_outputs
    ],
)

# Visualize results
view = py3Dmol.view(viewergrid=(1,3))

prepared_proteins = map(lambda x: Path(x), glob(str(client.workspace / "objects" / "*.pdb")))

for (i, file) in enumerate(prepared_proteins):
    with open(file, "r") as f:
        view.addModel(f.read(), "pdb", viewer=(0,i))
        view.setStyle({"cartoon": {"color": "spectrum"}}, viewer=(0,i))
        view.zoomTo(viewer=(0,i))
view.show()

# Check failures
for instance_id, (status, name, count) in (await client.status()).items():
    if status.value == "FAILED":
        async for log_page in client.logs(instance_id, "stderr"):
            for log in log_page:
                print(log)